### Premier League data Plotted using APis
 Using an api from football-data.org,
 with auth token: de27c6f949e040418ee4fcb5206ca657
 I have plotted data from that api, getting tables and sorting them

In [20]:
import requests
url = "https://api.football-data.org/v4/competitions/PL/matches"
headers = {"X-Auth-Token": "de27c6f949e040418ee4fcb5206ca657"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}, {response.text}")

In [21]:
import pandas as pd
df = pd.DataFrame(data['matches'])
df.head(3)


,area,competition,season,id,utcDate,status,matchday,stage,group,lastUpdated,homeTeam,awayTeam,score,odds,referees
0,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 2287, 'startDate': '2024-08-16', 'endDa...",497410,2024-08-16T19:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-03-26T00:21:15Z,"{'id': 66, 'name': 'Manchester United FC', 'sh...","{'id': 63, 'name': 'Fulham FC', 'shortName': '...","{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 11446, 'name': 'Robert Jones', 'type':..."
1,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 2287, 'startDate': '2024-08-16', 'endDa...",497411,2024-08-17T11:30:00Z,FINISHED,1,REGULAR_SEASON,None,2025-03-26T00:21:15Z,"{'id': 349, 'name': 'Ipswich Town FC', 'shortN...","{'id': 64, 'name': 'Liverpool FC', 'shortName'...","{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 11396, 'name': 'Tim Robinson', 'type':..."
2,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 2287, 'startDate': '2024-08-16', 'endDa...",497412,2024-08-17T14:00:00Z,FINISHED,1,REGULAR_SEASON,None,2025-03-26T00:21:15Z,"{'id': 57, 'name': 'Arsenal FC', 'shortName': ...","{'id': 76, 'name': 'Wolverhampton Wanderers FC...","{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,"[{'id': 23568, 'name': 'Jarred Gillett', 'type..."


 Selecting the name of a team alone from the "homeTeam" data

In [22]:
df['home_team_name'] = df['homeTeam'].apply(lambda x: x['name'] if isinstance(x, dict) else None)
df['away_team_name'] = df['awayTeam'].apply(lambda x: x['name'] if isinstance(x, dict) else None)

df['home_goals'] = df['score'].apply(lambda x: x['fullTime']['home'] if isinstance(x, dict) and 'fullTime' in x else None)
df['away_goals'] = df['score'].apply(lambda x: x['fullTime']['away'] if isinstance(x, dict) and 'fullTime' in x else None)


df['utcDate'] = pd.to_datetime(df['utcDate'])

# Format as 'Day, Month Date, Year - HH:MM AM/PM'
df['match_date'] = df['utcDate'].dt.strftime('%A, %B %d, %Y - %I:%M %p')


Plotting the matches in order of most recent

In [23]:

df_pl = df[df['status'] == 'FINISHED'].copy()

df_pl= df_pl[['home_team_name','status', 'away_team_name', 'home_goals', 'away_goals','match_date']]

df_pl['match_date'] = pd.to_datetime(df_pl['match_date'], format='%A, %B %d, %Y - %I:%M %p')
df_pl = df_pl.sort_values(by='match_date', ascending=False)

df_pl.head(10)


,home_team_name,status,away_team_name,home_goals,away_goals,match_date
288,Leicester City FC,FINISHED,Manchester United FC,0.0,3.0,2025-03-16 19:00:00
287,Fulham FC,FINISHED,Tottenham Hotspur FC,2.0,0.0,2025-03-16 13:30:00
286,Arsenal FC,FINISHED,Chelsea FC,1.0,0.0,2025-03-16 13:30:00
285,AFC Bournemouth,FINISHED,Brentford FC,1.0,2.0,2025-03-15 17:30:00
284,Southampton FC,FINISHED,Wolverhampton Wanderers FC,1.0,2.0,2025-03-15 15:00:00
283,Manchester City FC,FINISHED,Brighton & Hove Albion FC,2.0,2.0,2025-03-15 15:00:00
282,Ipswich Town FC,FINISHED,Nottingham Forest FC,2.0,4.0,2025-03-15 15:00:00
281,Everton FC,FINISHED,West Ham United FC,1.0,1.0,2025-03-15 15:00:00
280,West Ham United FC,FINISHED,Newcastle United FC,0.0,1.0,2025-03-10 20:00:00
279,Manchester United FC,FINISHED,Arsenal FC,1.0,1.0,2025-03-09 16:30:00


### To Get Standings 
to get the top 10 on the Premier League table

In [24]:
import requests
url = "https://api.football-data.org/v4/competitions/PL/standings"
headers = {"X-Auth-Token": "de27c6f949e040418ee4fcb5206ca657"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}, {response.text}")
# Extract standings data
standings = data['standings'][0]['table']  # The first element contains league table data

# Convert to DataFrame
df = pd.DataFrame(standings)


In [25]:
df['team_name'] = df['team'].apply(lambda x: x['name'] if isinstance(x, dict) else None)


df_standings= df[['position','team_name','playedGames','won','draw','lost','points','goalsFor','goalsAgainst','goalDifference']]

df_standings.head(10)

,position,team_name,playedGames,won,draw,lost,points,goalsFor,goalsAgainst,goalDifference
0,1,Liverpool FC,29,21,7,1,70,69,27,42
1,2,Arsenal FC,29,16,10,3,58,53,24,29
2,3,Nottingham Forest FC,29,16,6,7,54,49,35,14
3,4,Chelsea FC,29,14,7,8,49,53,37,16
4,5,Manchester City FC,29,14,6,9,48,55,40,15
5,6,Newcastle United FC,28,14,5,9,47,47,38,9
6,7,Brighton & Hove Albion FC,29,12,11,6,47,48,42,6
7,8,Fulham FC,29,12,9,8,45,43,38,5
8,9,Aston Villa FC,29,12,9,8,45,41,45,-4
9,10,AFC Bournemouth,29,12,8,9,44,48,36,12
